<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# Pub/Sub API (python)

In this notebook, you will learn how to use the [__Pub/Sub API__](https://googleapis.dev/python/pubsub/latest/index.html) for python.

In particular you will learn how to:
    
- Create/Delete a topic
- Listing all the available topics
- Creatie/Delete a subscriptions
- Listing all the available subscriptions
- Publish text messages
- Receiving text messages

### Install the libraries

In [1]:
pip install -q google-cloud

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -q google-cloud-pubsub

Note: you may need to restart the kernel to use updated packages.


### Setup the environment

In [3]:
# Import the python libraries

import os
from google.cloud import pubsub_v1


In [4]:
# Import the .json credentials and set the environment variable (TODO:student)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./credentials.json"

## Publisher

__Note__: in order to avoid conflicts, please name the topic_id as "\<surname\>-python-topic"

In [5]:
# Set the topic paramenters (TODO:student) 

project_id = "preparazione-lezione-gcp"
topic_id = "python-topic"

In [6]:
# Create a new topic 

publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(project_id, topic_id)

topic = publisher.create_topic(request={"name": topic_path})

print("Created topic: {}".format(topic.name))

Created topic: projects/preparazione-lezione-gcp/topics/python-topic


In [7]:
# List the topics to check that has been created

publisher = pubsub_v1.PublisherClient()
project_path = f"projects/{project_id}"

for topic in publisher.list_topics(request={"project": project_path}):
    print(topic)

name: "projects/preparazione-lezione-gcp/topics/heartratedata"

name: "projects/preparazione-lezione-gcp/topics/python-topic-json"

name: "projects/preparazione-lezione-gcp/topics/python-topic"



## Subscriber

__Note__: in order to avoid conflicts, please name the subscription_id as "\<surname\>-python-pull-subscription"

In [8]:
# Set the subscriber parameters (TODO:student)

subscription_id = "python-pull-subscription"

In [9]:
# Create a Pull subscription (Pull is default if not specified)

publisher = pubsub_v1.PublisherClient()
subscriber = pubsub_v1.SubscriberClient()
topic_path = publisher.topic_path(project_id, topic_id)
subscription_path = subscriber.subscription_path(project_id, subscription_id)

with subscriber:
    subscription = subscriber.create_subscription(
        request={"name": subscription_path, "topic": topic_path}
    )

print(f"Subscription created: {subscription}")

Subscription created: name: "projects/preparazione-lezione-gcp/subscriptions/python-pull-subscription"
topic: "projects/preparazione-lezione-gcp/topics/python-topic"
push_config {
}
ack_deadline_seconds: 10
message_retention_duration {
  seconds: 604800
}
expiration_policy {
  ttl {
    seconds: 2678400
  }
}



In [10]:
# List the subscriptions to check that has been screated

subscriber = pubsub_v1.SubscriberClient()
project_path = f"projects/{project_id}"

with subscriber:
    for subscription in subscriber.list_subscriptions(
        request={"project": project_path}
    ):
        print(subscription.name)

projects/preparazione-lezione-gcp/subscriptions/heartratedata
projects/preparazione-lezione-gcp/subscriptions/python-subscription-json
projects/preparazione-lezione-gcp/subscriptions/python-pull-subscription


## Pub/Sub in action (text data)

In [11]:
# Publish a text message

publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(project_id, topic_id)

for n in range(1, 10):
    data = "Hello. This is a text message 😀. Message number {}".format(n)
    # Data must be a bytestring
    data = data.encode("utf-8")
    # When you publish a message, the client returns a future.
    future = publisher.publish(topic_path, data)
    print(future.result())

print(f"Published messages to {topic_path}.")

2078200055411068
2078195048496154
2078215006989526
2078208592704296
2078209252101112
2078201497002651
2078208639212891
2078208795051212
2078200308329642
Published messages to projects/preparazione-lezione-gcp/topics/python-topic.


In [12]:
# Retrieve(pull) the messages from the subscriber

# Number of seconds the subscriber should listen for messages
timeout = 5.0


from concurrent.futures import TimeoutError

subscriber = pubsub_v1.SubscriberClient()
subscription_path = subscriber.subscription_path(project_id, subscription_id)

def callback(message):
    print(f"Received {message.data.decode('utf-8')}.")
    message.ack()

streaming_pull_future = subscriber.subscribe(subscription_path, callback=callback)
print(f"Listening for messages on {subscription_path}..\n")

# Wrap subscriber in a 'with' block to automatically call close() when done.
with subscriber:
    try:
        # When `timeout` is not set, result() will block indefinitely,
        # unless an exception is encountered first.
        streaming_pull_future.result(timeout=timeout)
    except TimeoutError:
        streaming_pull_future.cancel()


Listening for messages on projects/preparazione-lezione-gcp/subscriptions/python-pull-subscription..

Received Hello. This is a text message 😀. Message number 9.
Received Hello. This is a text message 😀. Message number 5.
Received Hello. This is a text message 😀. Message number 4.
Received Hello. This is a text message 😀. Message number 8.
Received Hello. This is a text message 😀. Message number 3.
Received Hello. This is a text message 😀. Message number 6.
Received Hello. This is a text message 😀. Message number 7.
Received Hello. This is a text message 😀. Message number 1.
Received Hello. This is a text message 😀. Message number 2.


## Cleaning Up

In [13]:
# Delete the subscription

subscriber = pubsub_v1.SubscriberClient()
subscription_path = subscriber.subscription_path(project_id, subscription_id)

with subscriber:
    subscriber.delete_subscription(request={"subscription": subscription_path})

print(f"Subscription deleted: {subscription_path}.")

Subscription deleted: projects/preparazione-lezione-gcp/subscriptions/python-pull-subscription.


In [14]:
# Delete the topic

publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(project_id, topic_id)

publisher.delete_topic(request={"topic": topic_path})

print("Topic deleted: {}".format(topic_path))

Topic deleted: projects/preparazione-lezione-gcp/topics/python-topic


##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) 2021 Quantia Consulting, srl. All rights reserved.